## Get Best-Performing Hyper Parameters

In [8]:
MONGO_COLLECTION = "SC_RE-RANKER_HYPER_PARAM_VD"

In [9]:
# for r in rows[-17:]:
#     print(str(r["asof"]))

In [10]:
cols = ["f1_score", "precision", "recall", "asof", "best_top_n", "C", "best_max_parses", 
    #"best_max_upd", 
    "max_update_items", 
    # "best_min_prob", "extractors", 
    "initial_weight", "loss_type",\
    #"min_feat_freq",
    "pa_type"]

In [11]:
import pandas as pd
import pymongo

client = pymongo.MongoClient(serverSelectionTimeoutMS=100, host="127.0.0.1")
db = client.metrics_causal_model_reranker

project = {
        "params": "$parameters",
        "micro_f1": "$MICRO_F1",
        "asof": "$asof",
        "_id": 1
    }
feats_pipeline = [{ "$project": project }]
rows = [row for row in db[MONGO_COLLECTION].aggregate(feats_pipeline)]

results = []
for r in rows:
    d = dict(r["params"])
    d.update(r["micro_f1"])
    d["asof"] = str(r["asof"])
    results.append(d)

In [12]:
df = pd.DataFrame(results)
df.tail(3)[["asof"] + cols]

,asof,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
61,2019-06-18 11:42:05.576000,0.805496,0.858949,0.758306,2019-06-18 11:42:05.576000,5,0.1,300,1,0.01,ml,1
62,2019-06-18 11:44:31.500000,0.797984,0.861978,0.742835,2019-06-18 11:44:31.500000,5,0.1,300,1,0.01,pb,2
63,2019-06-18 11:46:59.731000,0.796624,0.859830,0.742075,2019-06-18 11:46:59.731000,5,0.1,300,1,0.01,ml,2


In [13]:
df = pd.DataFrame(results)
df = df.sort_values(by="f1_score",ascending=False)
df[cols].head(5)

,f1_score,precision,recall,asof,best_top_n,C,best_max_parses,max_update_items,initial_weight,loss_type,pa_type
41,0.808089,0.856129,0.765153,2019-06-18 10:15:07.357000,3,0.0100,300,1,0.01,ml,1
39,0.807661,0.855805,0.764646,2019-06-18 10:01:40.165000,3,0.0025,300,1,0.01,ml,2
57,0.807500,0.862536,0.759067,2019-06-18 11:32:12.492000,5,0.0100,300,1,0.01,ml,1
37,0.806882,0.850914,0.767182,2019-06-18 09:47:53.638000,3,0.0025,300,1,0.01,ml,1
3,0.806736,0.868421,0.753234,2019-06-18 07:56:39.350000,1,0.0005,300,1,0.01,ml,2


In [14]:
# Best ones that are not 2
# df[df.best_top_n != 2][cols].head()